In [14]:
import 决策树可视化 as treeplotter_small_size
import numpy as np
from collections import Counter
import math
from sklearn.datasets import load_iris
iris_dataset = load_iris()
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =train_test_split(iris_dataset['data'], iris_dataset['target'],
                                                   test_size=0.5,random_state=1)

In [15]:
def change_target_names(dataset):
    y=[]
    for i in dataset:
        if i ==0:
            y.append('setosa')
        elif i ==1:
             y.append('versicolor')
        elif i ==2:
             y.append('virginica')
    return y
def Change(dataset):
    X=np.ones((dataset.shape[0],dataset.shape[1]))
    for i1 in range(dataset.shape[1]):
        train=X_train[:,i1]
        min=np.min(train)
        size=(np.max(train)-np.min(train))/10
        for i2 in range(dataset.shape[0]):
            if i1==0 or i1==3 or i1==1:
                if dataset[i2,i1] <=min+3.3*size:
                    X[i2,i1]=0
                elif min+3.3*size< dataset[i2,i1] <=min+6.6*size:
                    X[i2,i1]=1
                else:
                    X[i2,i1]=2
                #a=(max-min)/3
                #0=min+1/3*a
                    
            elif i1==2:
                if dataset[i2,i1] <=min+2.5*size:
                    X[i2,i1]=0
                elif min+2.5*size < dataset[i2,i1] <=min+5*size:
                    X[i2,i1]=1
                elif min+5*size < dataset[i2,i1] <=min+7.5*size:
                    X[i2,i1]=2
                elif min+7.5*size < dataset[i2,i1] <=min+11*size:
                    X[i2,i1]=3
                    
            elif i1==1:
                if dataset[i2,i1] <=min+5*size:
                    X[i2,i1]=0
                else:
                    X[i2,i1]=1            
    return X.astype(np.int64)

def New_change():
    H=[]
    for i in Change(X_train):
        H.append(list(i))
    y=list(change_target_names(y_train))
    new_data=[]
    w=0
    for data in H:
        data.append(y[w])
        new_data.append(data)
        w+=1
    return new_data
New_data=New_change()

In [16]:
import math
import operator
 
def calcShannonEnt(dataset):
    numEntries = len(dataset)#列数
    labelCounts = {}
    for featVec in dataset:
        currentLabel = featVec[-1]
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel] +=1
         
    shannonEnt = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key])/numEntries
        shannonEnt -= prob*math.log(prob, 2)
    return shannonEnt
     
def CreateDataSet(dataset):
    #labels = ['0','1','2','3']
    labels = ['sepal length','sepal width','petal length','petal width']
    return dataset, labels

def splitDataSet(dataSet, axis, value):
    retDataSet = []
    for featVec in dataSet:
        if featVec[axis] == value:
            reducedFeatVec = featVec[:axis]
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
     
    return retDataSet

def chooseBestFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0]) - 1      #获取当前数据集的特征个数，最后一列是分类标签
    baseEntropy = calcShannonEnt(dataSet)  #计算当前数据集的信息熵
    bestInfoGain = 0.0; bestFeature = -1   #初始化最优信息增益和最优的特征
    for i in range(numFeatures):        #遍历每个特征iterate over all the features
        featList = [example[i] for example in dataSet]#获取数据集中当前特征下的所有值
        uniqueVals = set(featList)       # 获取当前特征值，例如outlook下有sunny、overcast、rainy
        newEntropy = 0.0
        for value in uniqueVals: #计算每种划分方式的信息熵
            subDataSet = splitDataSet(dataSet, i, value)
            prob = len(subDataSet)/float(len(dataSet))
            newEntropy += prob * calcShannonEnt(subDataSet)
        infoGain = baseEntropy - newEntropy     #计算信息增益
        if (infoGain > bestInfoGain):       #比较每个特征的信息增益，只要最好的信息增益
            bestInfoGain = infoGain         #if better than current best, set to best
            bestFeature = i
    return bestFeature                      #returns an integer


 
def majorityCnt(classList):
    classCount ={}
    for vote in classList:
        if vote not in classCount.keys():
            classCount[vote]=0
        classCount[vote]=1
    sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)
    return sortedClassCount[0][0]
  
def createTree(dataSet, labels):
    classList = [example[-1] for example in dataSet]
    if classList.count(classList[0])==len(classList):
        return classList[0]
    if len(dataSet[0])==1:
        return majorityCnt(classList)
    bestFeat = chooseBestFeatureToSplit(dataSet)
    bestFeatLabel = labels[bestFeat]
    myTree = {bestFeatLabel:{}}
    del(labels[bestFeat])
    featValues = [example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[:]
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet, bestFeat, value), subLabels)
    return myTree

In [17]:
int(np.random.uniform(low=0, high=3, size=None))

1

In [18]:
#TrainData(训练数据)；forest_size（生成多少棵树）；ratio（每棵树抽取的比例）
def Random_forest(TrainData,forest_size,ratio):
    tree_set=[]
    z=[]#存放生成每棵树的数据引索号
    for i in range(forest_size):#生成多少棵树
        #从len(TrainData)[0]个数据中，抽取ratio*TrainData_X[0]个数据
        #生成每棵树的数据引索号
        a1= np.random.choice(a=len(TrainData), size=int(ratio*len(TrainData)), replace=False, p=None)
        z.append(a1)
        New_dataset=[]#生成每棵树的数据
        for a in a1:
            New_dataset.append(TrainData[a])
        myDat,labels = CreateDataSet(New_dataset)
        iris=createTree(myDat,labels)
        #tree_set存放所有决策树
        tree_set.append(iris)
    return tree_set

In [45]:
#TrainData(训练数据)；forest_size（生成多少棵树）；ratio（每棵树抽取的比例）
def Random_forest(TrainData,forest_size,ratio):
    treedata_set=[]
    z=[]#存放生成每棵树的数据引索号
    for i in range(forest_size):#生成多少棵树
        #从len(TrainData)[0]个数据中，抽取ratio*TrainData_X[0]个数据
        #生成每棵树的数据引索号
        a1= np.random.choice(a=len(TrainData), size=int(ratio*len(TrainData)), replace=False, p=None)
        z.append(a1)
        New_dataset=[]#生成每棵树的数据
        for a in a1:
            New_dataset.append(TrainData[a])
        treedata_set.append(New_dataset)
    tree_set=[]
    for tree in treedata_set:
        C45_iris=createTree(tree,['sepal length','sepal width','petal length','petal width'])
        tree_set.append(C45_iris)
    return tree_set

In [46]:
Random_forest(New_data,30,0.5)

[{'petal length': {0: 'setosa',
   1: 'versicolor',
   2: {'sepal length': {0: 'virginica',
     1: {'petal width': {1: 'versicolor', 2: 'virginica'}},
     2: 'versicolor'}},
   3: 'virginica'}},
 {'sepal width': {0: {'petal width': {1: 'versicolor', 2: 'virginica'}},
   1: {'sepal length': {0: 'setosa',
     1: {'petal width': {1: {'petal length': {'versicolor': 'versicolor',
         'virginica': 'virginica'}},
       2: 'virginica'}},
     2: {'petal length': {2: 'versicolor', 3: 'virginica'}}}},
   2: 'setosa'}},
 {'petal length': {0: 'setosa',
   1: 'versicolor',
   2: {'sepal length': {0: 'virginica',
     1: {'petal width': {1: 'versicolor',
       2: {'sepal width': {0: 'virginica', 1: 'virginica'}}}},
     2: 'versicolor'}},
   3: 'virginica'}},
 {'petal length': {0: 'setosa',
   1: 'versicolor',
   2: {'petal width': {1: {'sepal width': {0: 'versicolor',
       1: {'sepal length': {1: 'virginica', 2: 'versicolor'}}}},
     2: 'virginica'}},
   3: 'virginica'}},
 {'petal leng

In [47]:
#输入三个变量（决策树，属性特征标签，测试的数据）
labels = ['sepal length','sepal width','petal length','petal width']
def classify(inputTree,featLables,testVec):
    firstStr=list(inputTree.keys())[0] #获取树的第一个特征属性
    secondDict=inputTree[firstStr] #树的分支，子集合Dict
    featIndex=featLables.index(firstStr) #获取决策树第一层在featLables中的位置
    for key in secondDict.keys():
        if testVec[featIndex]==key:
            global classLabel   #声明
            if type(secondDict[key]).__name__=='dict':
                classLabel=classify(secondDict[key],featLables,testVec)
            else:classLabel=secondDict[key]
    return classLabel

In [58]:
data=Random_forest(New_data,30,0.4)

In [59]:
def Random_forest_classify(TrainData,forest_size,ratio,data):
    Classification_result_set=[]
    for i in Random_forest(TrainData,forest_size,ratio):
        Classification_result_set.append(classify(i,['sepal length','sepal width','petal length','petal width'],data))
    return Counter(Classification_result_set).most_common(1)[0][0]

In [60]:
Random_forest_classify(New_change(),30,0.5,[1,1,1,1])

'versicolor'

In [ ]:
def ID3(X):
    test_data=Change(X)   #使用临时变量得到输入数组的大小
    y_pred=np.ones((test_data.shape[0]))
    w=0
    for data in test_data:
        pred=Random_forest_classify(New_change(),30,0.5,data)
        if pred == "setosa":
            y_pred[w]=0
        elif pred == "versicolor":
            y_pred[w]=1
        elif pred == "virginica":
            y_pred[w]=2
        w+=1
    return y_pred

In [62]:
def Test_set_score(Test_data,Result,Papers):
    #print("Test set score: {:.2f}".format(np.mean(Result == Papers)))
    length=len(Result)
    #for i in range(length):
        #if Result[i] != Papers[i]:
            #print("X_test{0}     y_pred:{1}    y_test:{2}".format(Test_data[i],Result[i],Papers[i]))
    return np.mean(Result == Papers)
Test_set_score(Change(X_test),ID3(X_test),y_test)

1.0

In [63]:
正确率1=[]
X_train, X_test, y_train, y_test =train_test_split(iris_dataset['data'], iris_dataset['target'],
                                                   test_size=0.6,random_state=i)
a=Test_set_score(X_test,ID3(X_test),y_test)
正确率1.append(a)
print(np.mean(正确率1))

0.944444444444


In [71]:
for qwe in [0.1,0.2,0.3,0.4,0.5,0.6,0.7]:
    for i in range(1,100):
        X_train, X_test, y_train, y_test =train_test_split(iris_dataset['data'], iris_dataset['target'],
                                                       test_size=qwe,random_state=i)
        a=Test_set_score(X_test,ID3(X_test),y_test)
        正确率1.append(a)
    print(np.mean(正确率1))

0.940877463806
0.942350669372
0.943510472179
0.94394609487
0.94427483653
0.94405744789
0.942895611702
